<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# Naas - Create Pipeline
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Create_Pipeline.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+-+Create+Pipeline:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naas #pipeline #jupyter #notebook #dataanalysis #workflow #streamline

**Author:** [Maxime Jublou](https://www.linkedin.com/in/maximejublou)

**Description:** This notebook is a guide that teaches you how to create a notebook pipeline using naas.

## Input

### Import libraries

In [ ]:
from naas.pipeline import (
    Pipeline,
    DummyStep,
    DummyErrorStep,
    NotebookStep,
    End,
    ParallelStep,
)

### Setup variables

In [ ]:
# LinkedIn
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
linkedin_url = "https://www.linkedin.com/in/maximejublou/"  # EXAMPLE "https://www.linkedin.com/in/XXXXXX/"
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine")
print("Output directory:", output_dir)

# Google Sheets
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1GDVJxBajYx7VR48yKGnSvMsEjzXvqI94hlmPXI5qEp8/edit#gid=0" or naas.secret.get("MY_AI_AVATAR_SPREADSHEET")
sheet_name = "CONTENT"

# Notification
email_to = "jeremy@naas.ai"

# Scheduler
cron = "0 8 * * *"

## Model

### Setup notebooks
This section is made to declare all the notebooks and create unique IDs that will be used in the pipeline. 

In [ ]:
extraction = DummyStep(
    "Get LinkedIn posts"
)
cleaning = DummyStep(
    "Send data to Google Sheets"
)
analytics = DummyStep(
    "Create analytics"
)  
distribution1 = DummyStep(
    "Send Email notification"
)
distribution2 = DummyStep(
    "Generate plugin"
)

## Output

### Run pipeline

In [ ]:
pipeline = Pipeline()

pipeline >> extraction >> cleaning >> analytics >> [distribution1, distribution2] >> End()

pipeline.run(outputs_path=os.path.join(output_dir, "pipeline_executions"))

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)